In [14]:
import os
import pdfquery
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceEmbeddings
import faiss
from langchain_groq import ChatGroq
from langchain.document_loaders import PyPDFLoader



In [15]:
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
print(GROQ_API_KEY)

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


In [16]:
llm = ChatGroq(model_name="deepseek-r1-distill-qwen-32b", api_key="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")

In [17]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")



In [18]:
def extract_text_from_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    return documents

In [22]:
raw=extract_text_from_pdf("IDS COURSE CONTENT.pdf")
raw

[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2022-11-25T09:21:07+00:00', 'author': 'NRCM', 'moddate': '2022-11-25T09:21:07+00:00', 'source': 'IDS COURSE CONTENT.pdf', 'total_pages': 98, 'page': 0, 'page_label': '1'}, page_content='INTRODUCTION TO DATA SCIENCE \n1 \nCSE NRCM          P.LAKSHMI PRASANNA(ASST.PROFESSOR) \nINTRODUCTION TO DATA SCIENCE \n LECTURE NOTES \nUNIT - 1 \nIntroduction to data science \nData science: \nData science is the domain of study that deals with vast volumes of data using modern tools and \ntechniques to find unseen patterns, derive meaningful information, and make business decisions. \nData science uses complex machine learning algorithms to build predictive models. \nThe data used for analysis can come from many different sources and presented in various \nformats. \nData science is about extraction, preparation, analysis, visualization, and maintenance of \ninformation. It is a cross disciplinary

In [23]:
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)


In [24]:
chunks = text_splitter.split_documents(raw)

In [25]:
from langchain.vectorstores import FAISS
vector_store = FAISS.from_documents(chunks, embeddings)


In [26]:
retriever = vector_store.as_retriever()

In [27]:
query="1990s and early 2000s: We can clearly"
results = vector_store.similarity_search(query, k=3)

In [28]:
results

[Document(id='8687e8f1-87a3-471a-8a69-bc7d9f6288b4', metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2022-11-25T09:21:07+00:00', 'author': 'NRCM', 'moddate': '2022-11-25T09:21:07+00:00', 'source': 'IDS COURSE CONTENT.pdf', 'total_pages': 98, 'page': 1, 'page_label': '2'}, page_content='INTRODUCTION TO DATA SCIENCE \n2 \nCSE NRCM          P.LAKSHMI PRASANNA(ASST.PROFESSOR) \nCommunicate: Data Reporting, Data Visualization, Business Intelligence, Decision Making. In \nthis final step, analysts prepare the analyses in easily readable forms such as charts, graphs, and \nreports. \nEvolution of Data Science: Growth & Innovation \nData science was born from the idea of merging applied statistics with computer science. The \nresulting field of study would use the extraordinary power of modern computing. Scientists \nrealized they could not only collect data and solve statistical problems but also use that data to \nsolve real-world problems and ma

In [30]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
import re

# Define the prompt template for questions
question_prompt_template = """
You are a helpful assistant that answers questions based on the content of a document. 
The document's context is as follows:
{context}

Provide only your final answer without any thinking process or intermediate steps:
{question}
"""

# Create the prompt templates
QUESTION_PROMPT = PromptTemplate(
    template=question_prompt_template,
    input_variables=["context", "question"]
)

# Initialize conversation memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Initialize the chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={'prompt': QUESTION_PROMPT},
    return_source_documents=False
)

def get_clean_answer(query: str) -> str:
    """
    Get only the final answer from the QA chain response, removing any thinking process.
    
    Args:
        query (str): The question to ask
        
    Returns:
        str: The clean answer only
    """
    response = qa_chain.invoke({"question": query})
    answer = response['answer']
    
    # Remove thinking process if present
    if "<think>" in answer.lower():
        # Extract everything after the last occurrence of thinking-related content
        clean_answer = re.split(r'<think>.*?</think>', answer, flags=re.IGNORECASE|re.DOTALL)[-1]
        return clean_answer.strip()
    
    return answer.strip()

# Example usage
query = "planning and managing end-to-end data architecture"
clean_answer = get_clean_answer(query)
print(clean_answer)

The role of a data architect in planning and managing end-to-end data architecture involves designing blueprints for data management, ensuring databases are integrated, centralized, and secure. They align the data strategy with business goals, identify appropriate data sources, and collaborate with cross-functional teams to ensure effective database systems.
